In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

1. Do your work for this exercise in a notebook named explore. Use the techniques in the lesson to explore the store item demand dataset and the opsd dataset.

2. For the store item demand data, you will need to choose a method of aggregating such that each observation is a unique date. For both datasets you should choose a method of splitting your data and only explore the training split.

In [2]:
opsd = pd.read_csv(r'/Users/hector/Documents/cached_csvs/opsd.csv')

In [3]:
item_demand = pd.read_csv(r'/Users/hector/Documents/cached_csvs/total_sales_data.csv')

In [1]:
#...